This file contains the implementation of ridge (L2) regression for mapping fMRI embeddings to CLIP and word2vec embedding

First setup baseline model (word2vec) (Not yet implemented)

In [ ]:


# semeval-2016-2017-task3-subtaskA-unannotated
model = api.load("glove-twitter-25")  # download the model and return as object ready for use
# model.most_similar("cat")

In [4]:
!pip install transformers

In [1]:
import tensorflow as tf

2023-11-23 12:49:25.729961: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from scipy import signal
from pathlib import Path
from functools import reduce
# from google.colab import drive
import pandas as pd
import pickle
from PIL import Image
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# drive.mount('/content/drive')
fMRI_folder = Path('./doi_10.5061_dryad.gt413__v1')

In [4]:
NUM_SUBJS = 8
subjects_fmri = [] #stores all 8 subject fmri np arrays

# fMRI_folder = Path('./doi_10.5061_dryad.gt413__v1')
# assert fMRI_folder.exists(), f"Foldder: {fMRI_folder} does not exist."

for subj_id in range(8):
    print("Subject:",subj_id)
#     fmri_file_name = str(subj_id) + '_masked_2d.npy'
#     fmri = np.load(fMRI_folder / fmri_file_name)
    fmri_file_name = str(subj_id) + '_smooth_nifti_4d.nii'
    fmri = nib.load(fMRI_folder / fmri_file_name)
    fmri = np.array(fmri.dataobj)
    assert isinstance(fmri, np.ndarray), f"Imported fmri_scan for subject {subj_id} is not of type numpy.ndarray"
    assert(fmri.ndim) == 4, f"Imported fmri_scan for subject {subj_id} is not 4 dimensional"
    subjects_fmri.append(fmri)


Subject: 0
Subject: 1
Subject: 2
Subject: 3
Subject: 4
Subject: 5
Subject: 6
Subject: 7


In [5]:
for x in subjects_fmri:
    print(np.shape(x))

(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)
(53, 60, 50, 1311)


In [6]:
mat_files = sorted(list(fMRI_folder.glob('subject_*.mat')))
sub_num = 0
matrices = []
for file in mat_files:
    mat_contents = sio.loadmat(file)
    matrix = np.zeros((np.shape(subjects_fmri[sub_num])[3],mat_contents["meta"][0][0][2][0][0],))
    fmri = subjects_fmri[int(str(file)[-5])-1]
    #print(np.shape(fmri))
    #print(mat_contents["meta"])

    print(mat_contents["meta"][0][0][3][0][0],mat_contents["meta"][0][0][4][0][0],mat_contents["meta"][0][0][5][0][0])
    for x in range(mat_contents["meta"][0][0][3][0][0]):
        for y in range(mat_contents["meta"][0][0][4][0][0]):
            for z in range(mat_contents["meta"][0][0][5][0][0]):
                col = mat_contents["meta"][0][0][7][x][y][z]
                matrix[:,col] = fmri[x-1][y-1][z-1][:]
    matrices.append(matrix)


53 60 50
51 60 49
53 59 47
53 59 44
51 59 45
49 56 51
52 60 49
52 60 48


In [7]:
feature_matrix = np.zeros((5176,195)) #stores the feature vectors as a row for each word
feature_names = [] #stores the names of all features in order
feature_types = {} #stores the types of features and all the names of the features for each type

features = sio.loadmat(fMRI_folder/'story_features.mat')
feature_count = 0
for feature_type in features['features'][0]:
    feature_types[feature_type[0][0]] = []
    if isinstance(feature_type[1][0], str):
        feature_types[feature_type[0][0]].append(feature_type[1][0])
        feature_names.append(feature_type[1][0])
    else:
        for feature in feature_type[1][0]:
            feature_types[feature_type[0][0]].append(feature[0])
            feature_names.append(feature[0])
    feature_matrix[:, feature_count:feature_count+feature_type[2].shape[1]] = feature_type[2] #adds the (5176xN) feature values to the feature matrix for the current feature group
    feature_count += feature_type[2].shape[1]

In [8]:
words_info = [] #stores tuples of (word, time, features) sorted by time appeared

mat_file = 'subject_1.mat' #only looks at the first subject file, somewhere it said all the timings were the same so this should be safe
mat_contents = sio.loadmat(fMRI_folder/mat_file)
# print(mat_content)
for count, row in enumerate(mat_contents['words'][0]):
    word_value = row[0][0][0][0]
    time = row[1][0][0]
    
    word_tuple = (word_value, time, feature_matrix[count,:])
    words_info.append(word_tuple)

In [9]:
chapter_nine_text = ""
for row in mat_contents['words'][0]:
    chapter_nine_text += row[0][0][0][0] + " "
# print(chapter_nine_text)

In [10]:
for count, subject in enumerate(subjects_fmri):
    print(count, np.shape(subject))

0 (53, 60, 50, 1311)
1 (53, 60, 50, 1311)
2 (53, 60, 50, 1311)
3 (53, 60, 50, 1311)
4 (53, 60, 50, 1311)
5 (53, 60, 50, 1311)
6 (53, 60, 50, 1311)
7 (53, 60, 50, 1311)


In [11]:
with open('fmri_indices', 'rb') as f:
    fmri_indices = pickle.load(f)

In [12]:
def hrf_alignment_2(num_words, words_info, NUM_SUBJS, subjects_fmri):
    #for each word, get the next 4 fMRI scans weighted by the gaussian window for each subject
    #then save the word fMRI scans for each subject and the words in an pandas file
    window = signal.windows.gaussian(16, std=1) #gaussian window for the 4 fMRI scans
    subject_words_dict = [{'file_name':[], 'word':[], 'time':[]} for i in range(8)]
    for word_count in words_info:
        word = word_count[0]
        time = word_count[1]
        #print(word, time)
        fmri_count = 0
        subject_scans = []
        for i in range(1,17):
            delay = 0.5*i #time after word was read
            try:
                curr_fmri_idx = fmri_indices.index((time + delay)/2) #checks if an fMRI scan happens at this time point
                weight = window[int(2*delay)-1]
                for count, subject in enumerate(subjects_fmri):
                    if fmri_count == 0:
                        subject_scans.append(weight*subject[:,:,:,curr_fmri_idx])
                    else:
                        subject_scans[count] += weight*subject[:,:,:,curr_fmri_idx]
                fmri_count += 1
            except Exception as e:
                #print(e)
                pass
        #print(fmri_count)
        if fmri_count == 4:
            for count, subject in enumerate(subjects_fmri):
                #save filename with (word, time) in file
                file_name = "./word_fmris/" + str(count) + "_subject_word_weighted_" + str(time) + ".pt"
                scan = torch.tensor(subject_scans[count])
                with open(file_name, 'wb') as f:
                    torch.save(scan, f)
                subject_words_dict[count]['file_name'].append(file_name)
                subject_words_dict[count]['word'].append(word)
                subject_words_dict[count]['time'].append(time)
        for count, subject in enumerate(subjects_fmri):
            df = pd.DataFrame(subject_words_dict[count])
            df.to_csv("./" + str(count) + "_subject_word_fmri_labels.csv", index=False)

Harrison's implementation of Gaussian weighting

In [13]:
def hrf_alignment(num_words, words_info, NUM_SUBJS, subjects_fmri):

    subjects_samples = [[] for i in range(NUM_SUBJS)] #stores lists of all the samples for each subject
    window = signal.windows.gaussian(16, std=1) #gaussian window for the 4 fMRI scans
    num_words = num_words
    word_count = 0
    while word_count < len(words_info) - 24:
        #gets the 4 input words, and the 4 consecutive words while verifying they were read in sequence
        scan_words = []
        start_time = words_info[word_count][1]
        in_sequence = True #tracks if the words are in sequence or not
        for i in range(num_words):
            word_info = words_info[word_count + i]
            if word_info[1] != start_time + 0.5*i:
                #if some of the words are not in sequence, skip forward 1 word after innter loop
                in_sequence = False
            scan_words.append(word_info[0])
        if not in_sequence:
            word_count +=1
            continue
        fmri_time = start_time #fMRI is taken at first word read
        fmri_index = fmri_time//2 #since a scan happens every two seconds, the index is the time divided by 2
        if not np.issubdtype(fmri_index, np.integer):
            #if the first word is not aligned with the fmri scan (i.e. its not the first word in a TR)
            word_count += 1
            continue
        break_found = False
        for count, subject in enumerate(subjects_fmri):
#             print("Subject:",count)
            #adds tuple of (fmri_scan, four words)
            for i in range(num_words): #time delay of reading word after fMRI scan
                delay = 0.5*i #delay after first word
                for j in range(1,5): #get next 4 fMRI scans
                    fMRI_delay = 2*j - delay #delay until jth next fMRI scan
                    weight = window[int(2*fMRI_delay)-1] #gets the gaussian weighting (16 points instead of 8 to allow for .5)
                    try:
                        curr_fmri_idx = fmri_indices.index((start_time + math.floor(i/4) + 2*j)//2) #gets the index of the jth next fMRI scan
                    except:
#                         print("break found at index:",(start_time + math.floor(i/4) + 2*j)//2)
                        break_found = True
                        break
                    #adds weight fMRI scan
                    if j == 1:
                        word_scan = weight*subject[:][curr_fmri_idx]
                    else:
                        word_scan += weight*subject[:][curr_fmri_idx]
                if break_found:
                    break
                if i == 0:
                    summed_weighted_scans = word_scan
                else:
                    summed_weighted_scans += word_scan
            if break_found:
                break
            subjects_samples[count].append((summed_weighted_scans/num_words, scan_words))
        if break_found:
            word_count += 1
            continue 
#         print("Created sample:")
#         print("\tScan time:", str(start_time))
#         print("\tInput words:", str(scan_words))
        #if successful, skip forward to the next set of 4 words
        word_count += 4
    return subjects_samples

    print("Total number of samples:", str(len(subjects_samples[0])))

In [14]:
def get_fMRI_embedding(subjects_samples):
    scaler = StandardScaler()

    flattened_shape = reduce(np.multiply, subjects_samples[0][0].shape)
    X_matrix = np.zeros((len(subjects_samples), flattened_shape))

    for idx, sample in enumerate(subjects_samples):
        tmp = sample[0]
        # Reshape the voxels 
        tmp = tmp.ravel()
        X_matrix[idx,:] = tmp
        
    # Apply voxelwise standardization        
    X_matrix = scaler.fit_transform(X_matrix)
    #print(np.shape(X_matrix))
    return X_matrix

In [15]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [16]:
import torch
from transformers import BertTokenizer, BertModel, DistilBertTokenizer,DistilBertModel
from sentence_transformers import SentenceTransformer, util

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
#% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


In [17]:
def get_SBERT_embedding(subject_sample):
    word_list = []
    context_length = 0

    for i in range(len(subject_sample)):
        context_length = len(subject_sample[i][1])
        word_list.append(subject_sample[i][1])
    new_word_list = [(' '.join(sublist)) for sublist in word_list]
    print(context_length)
    model = SentenceTransformer('bert-base-nli-mean-tokens')
    x = model.encode(new_word_list,convert_to_tensor=True)
    return x

In [18]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

#module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
#model = hub.load(module_url)
#print ("module %s loaded" % module_url)
##def embed(input):
 # return model(input)


In [22]:
module_url = "https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/large/versions/2"
model_USE = hub.load(module_url)

In [23]:
def get_USE_embedding(subject_sample):
    word_list = []
    context_length = 0

    for i in range(len(subject_sample)):
        context_length = len(subject_sample[i][1])
        word_list.append(subject_sample[i][1])
    new_word_list = [(' '.join(sublist)) for sublist in word_list]
    #print(new_word_list)
    #print(context_length)
    #model = hub.load(module_url)
    #print ("module %s loaded" % module_url)
    sentence_embeddings = model_USE(new_word_list)

    return np.array(sentence_embeddings)

In [59]:
y_matrix = get_USE_embedding(subjects_samples[0])


In [58]:
print(np.shape(y_matrix))

(1235, 512)


Jai's Implementation of clip embedding

In [24]:
def get_clip_embedding(subject_sample, model,processor):
    word_list = []
    for i in range(len(subject_sample)):
        word_list.append(subject_sample[i][1])
    new_word_list = [' '.join(sublist) for sublist in word_list]

    inputs = processor(text=new_word_list, images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        # text_embeds is in the same space as the given image
        text_embeds = outputs['text_embeds']

        # pooler_output is just the text embedding 
        # This is what we want to use I think 
        pooler_output = outputs['text_model_output']['pooler_output']
#         print(pooler_output.numpy().shape)
    #return pooler_output
    return text_embeds

In [25]:
# split data into train and test

def split_data(X_matrix, Y_matrix):
    X_train, X_test, y_train, y_test = train_test_split(X_matrix, y_matrix,
                                                        test_size=0.20,
                                                        random_state=1234,
                                                        shuffle=False)
    return X_train, X_test, y_train, y_test

In [26]:
# Train Ridge Regression Model

def train_model(X_train, y_train):
    
    ridge_model = Ridge(alpha=1.0)
    model = ridge_model.fit(X_train, y_train)
    
    return model


In [27]:
# Evaluate the model

def evaluate_model(model, X_test, y_test, random_text_embedding):
    correct_predictions = 0

    # Predict text embedding from test fMRI data
    y_pred = model.predict(X_test)
#     print(np.shape(y_pred))

    # Compute cosine similarity
    prediction_similarity_true = cosine_similarity(y_test, y_pred)
    prediction_similarity_random = cosine_similarity(random_text_embedding, y_pred)
#     print(np.shape(prediction_similarity_true), np.shape(prediction_similarity_true))
    
     # Count the occurrences of "1" in the first column    
    for i in range(X_test.shape[0]):
        if prediction_similarity_true[i].mean() >= prediction_similarity_random[i].mean():
            correct_predictions +=1

    # Calculate the accuracy
    accuracy = (correct_predictions / X_test.shape[0])
    
    return accuracy


In [28]:
def obtain_two_way_class_result(model, X_test, y_test, y_matrix, batches):
    accuracy = []
    for i in range(batches):
        idx = np.arange(len(y_test))
        np.random.shuffle(idx)

        random_text_embedding = y_matrix[idx]

        test_accuracy = evaluate_model(model, X_test, y_test, random_text_embedding)
        accuracy.append(test_accuracy)

    avg_accuracy = sum(accuracy)/ batches
    
    #print(accuracy)
    #print(f"Average Accuracy: {avg_accuracy * 100:.2f}%")
    
    return avg_accuracy

In [34]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold

In [30]:
context_lengths = [4,8,12,16,20]
NUM_SUBJS = 8

for length in context_lengths:
    # Step 1: Obtain aligned fMRI embedding with words
    print(f'Obtaining aligned fMRI scan for context length: {length}---------------------------------------------')
    subjects_samples = hrf_alignment(length, words_info, NUM_SUBJS, matrices)
    #y_matrix = get_BERT_embedding(subjects_samples[0])
    y_matrix = get_USE_embedding(subjects_samples[0])

    #y_matrix = get_clip_embedding(subjects_samples[0],model,processor)

    # Step 2: Obtain text and fMRI embedding
    # get embeddings for subject 1
    for subject in range(NUM_SUBJS):
        #print(f'---------Obtaining clip and fMRI embedding for subject {subject}-----------------')
        #y_matrix = get_clip_embedding(subjects_samples[subject],model,processor)
        X_matrix = get_fMRI_embedding(subjects_samples[subject])
        #print(f'Subject {subject} \n fMRI Embedding Shape: {np.shape(X_matrix)} \n Text Embedding Shape: {np.shape(y_matrix)}')

        # Split data for training
        #print(f'---------Spliting data for subject {subject}-----------------')
        #X_train, X_test, y_train, y_test = split_data(X_matrix, y_matrix)        #print(f'---------Training started for subject {subject}-----------------')
        # Train model
        model_ = train_model(X_train, y_train)
        
        # Evaluate model
        #print(f'---------Getting two way classification accuracy for subject {subject}-----------------\n')

        batches = 8
        accuracy = obtain_two_way_class_result(model_, X_test, y_test, y_matrix, batches)
        
        print(f'{accuracy*100:.2f}')
        
##use2



Obtaining aligned fMRI scan for context length: 4---------------------------------------------


KeyboardInterrupt: 

In [37]:
context_lengths = [4,8,12,16,20]
NUM_SUBJS = 8

for length in context_lengths:
    # Step 1: Obtain aligned fMRI embedding with words
    print(f'Obtaining aligned fMRI scan for context length: {length}---------------------------------------------')
    subjects_samples = hrf_alignment(length, words_info, NUM_SUBJS, matrices)
    #y_matrix = get_BERT_embedding(subjects_samples[0])
    y_matrix = get_USE_embedding(subjects_samples[0])

    #y_matrix = get_clip_embedding(subjects_samples[0],model,processor)

    # Step 2: Obtain text and fMRI embedding
    # get embeddings for subject 1
    for subject in range(NUM_SUBJS):
        #print(f'---------Obtaining clip and fMRI embedding for subject {subject}-----------------')
        #y_matrix = get_clip_embedding(subjects_samples[subject],model,processor)
        X_matrix = get_fMRI_embedding(subjects_samples[subject])
        #print(f'Subject {subject} \n fMRI Embedding Shape: {np.shape(X_matrix)} \n Text Embedding Shape: {np.shape(y_matrix)}')
        #cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
        # Split data for training
        #print(f'---------Spliting data for subject {subject}-----------------')
        #X_train, X_test, y_train, y_test = split_data(X_matrix, y_matrix)
        
        #print(f'---------Training started for subject {subject}-----------------')
        # Train model
        model_ = Ridge(alpha=1.0)
        kf = KFold(n_splits=5)
        # Evaluate model
        #print(f'---------Getting two way classification accuracy for subject {subject}-----------------\n')
        print(cross_val_score(model_, X_matrix, y_matrix, cv=kf,scoring="accuracy"))
        batches = 8
        accuracy = obtain_two_way_class_result(model_, X_test, y_test, y_matrix, batches)
        
        print(f'{accuracy*100:.2f}')



Obtaining aligned fMRI scan for context length: 4---------------------------------------------


/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 220, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/Caskroom/miniforge

[nan nan nan nan nan]


/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:839: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 214, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 220, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/Caskroom/miniforge

NameError: name 'X_test' is not defined